In [ ]:
import pandas as pd
import numpy as np

- Carga del archivo business.pkl

In [ ]:
ruta_pkl = r"C:\Users\Sofita\Documents\Proyecto Final\Yelp\\business.pkl" 
data = pd.read_pickle(ruta_pkl)

In [3]:
ruta_csv = r"C:\Users\Sofita\Documents\Proyecto Final\Yelp\\business.csv"  
data.to_csv(ruta_csv, index=False)  

In [42]:
business = data.copy()

- Carga del archivo 'review.jason'

In [ ]:
# Leer el archivo JSON en fragmentos
chunk_size = 100000  # Ajusta el tamaño del fragmento según tu memoria disponible
chunks = pd.read_json(r'C: ... \\Yelp\\review.json', chunksize=chunk_size, lines=True)

# Crear un archivo CSV vacío y escribir los fragmentos
csv_file = r'C: ... \\Yelp\\reviews_output.csv'

# Usar 'mode' para evitar sobreescribir el archivo
for i, chunk in enumerate(chunks):
    if i == 0:
        # Escribir el encabezado solo en el primer fragmento
        chunk.to_csv(csv_file, mode='w', index=False, header=True)
    else:
        # Escribir solo las filas para los fragmentos subsiguientes
        chunk.to_csv(csv_file, mode='a', index=False, header=False)

print(f"Archivo CSV guardado en: {csv_file}")

Archivo CSV guardado en: C:\\Users\\Sofita\\Documents\\Proyecto Final\\Yelp\\reviews_output.csv


In [ ]:
csv_file = r'C:\\ . . . \\Yelp\\reviews_output.csv'
reviews = pd.read_csv(csv_file)

In [6]:
reviews_yelp = reviews.copy()

In [70]:
reviews_yelp.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4,1,0,1,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15


- Bussiness

In [10]:
business.columns

Index(['business_id', 'name', 'address', 'city', 'state', 'postal_code',
       'latitude', 'longitude', 'stars', 'review_count', 'is_open',
       'attributes', 'categories', 'hours', 'business_id', 'name', 'address',
       'city', 'state', 'postal_code', 'latitude', 'longitude', 'stars',
       'review_count', 'is_open', 'attributes', 'categories', 'hours'],
      dtype='object')

- Función para eliminar columnas duplicadas

In [43]:
def eliminar_columnas_duplicadas(df):
    """
    Elimina columnas duplicadas manteniendo solo la primera aparición.

    Args:
    - df (pd.DataFrame): DataFrame que contiene las columnas duplicadas.

    Returns:
    - pd.DataFrame: DataFrame sin columnas duplicadas.
    """
    columnas_repetidas = df.columns[df.columns.duplicated()].unique()
    if len(columnas_repetidas) > 0:
        print(f"Columnas duplicadas encontradas: {list(columnas_repetidas)}")
    return df.loc[:, ~df.columns.duplicated()]


In [44]:
business = eliminar_columnas_duplicadas(business)

Columnas duplicadas encontradas: ['business_id', 'name', 'address', 'city', 'state', 'postal_code', 'latitude', 'longitude', 'stars', 'review_count', 'is_open', 'attributes', 'categories', 'hours']


- Convertir las categorías de la columna 'categories' separadas por comas en listas

In [46]:
import numpy as np

def transformar_categorias_a_listas(df, columna):
    """
    Transforma los elementos de una columna que contienen categorías separadas por comas
    en listas. Maneja valores nulos y tipos no cadena de forma segura.
    
    Args:
        df (pd.DataFrame): DataFrame que contiene los datos.
        columna (str): Nombre de la columna a transformar.
    
    Returns:
        pd.DataFrame: DataFrame con la columna transformada.
    """
    if columna not in df.columns:
        raise ValueError(f"La columna '{columna}' no existe en el DataFrame.")
    
    def convertir_a_lista(x):
        # Si el valor es una cadena válida, dividir por comas y espacios
        if isinstance(x, str):
            return x.split(', ')
        # Si es NaN o None, retornarlo como está
        elif pd.isna(x):
            return np.nan
        # En otros casos, retornar el valor original
        return x
    
    # Aplicar la transformación a toda la columna
    df[columna] = df[columna].apply(convertir_a_lista)
    return df



In [47]:
business = transformar_categorias_a_listas(business, 'categories')

In [48]:
business['categories'].head()

0    [Doctors, Traditional Chinese Medicine, Naturo...
1    [Shipping Centers, Local Services, Notaries, M...
2    [Department Stores, Shopping, Fashion, Home & ...
3    [Restaurants, Food, Bubble Tea, Coffee & Tea, ...
4                          [Brewpubs, Breweries, Food]
Name: categories, dtype: object

- Filtrar filas donde las listas en la columna 'categories' contienen la palabra 'Restaurant'

In [61]:
def filtrar_por_palabras(df, columna, palabras):
    """
    Filtra filas de un DataFrame donde los valores en una columna (listas o strings)
    contienen al menos una palabra específica de una lista dada. Luego, reinicia los índices.

    Args:
        df (pd.DataFrame): El DataFrame a procesar.
        columna (str): El nombre de la columna.
        palabras (list): Lista de palabras a buscar.

    Returns:
        pd.DataFrame: Un nuevo DataFrame con las filas filtradas y los índices reiniciados.
    """
    if columna not in df.columns:
        raise ValueError(f"La columna '{columna}' no existe en el DataFrame.")
    
    if not isinstance(palabras, list):
        raise ValueError("El argumento 'palabras' debe ser una lista de palabras.")
    
    # Filtrar filas
    filtrado = df[df[columna].apply(
        lambda x: (
            isinstance(x, list) and any(palabra in x for palabra in palabras)
        ) or (
            isinstance(x, str) and x in palabras
        )
    )]
    
    # Reiniciar índices
    return filtrado.reset_index(drop=True)


In [62]:
palabras = ['Restaurants']

business_rest = filtrar_por_palabras(business, 'categories', palabras)

In [ ]:
business_rest.info()

- Realizar el merge conservando todos los datos de business_rest y solo los coincidentes de reviews_yelp

In [71]:
restaurantes = business_rest.merge(reviews_yelp, on='business_id', how='left')

In [ ]:
restaurantes.to_csv('restaurantes.csv', index = False)

## Filtros

In [ ]:
# Se crea una copia para no afectar el original
prueba_restaurantes = restaurantes.copy()

In [ ]:
# Convertir a Dask DataFrame
ddf = dd.from_pandas(prueba_restaurantes, npartitions=4)  

In [ ]:
# Definir las regiones y sus respectivos estados
regiones = {
    'I-Noreste': ['CT', 'ME', 'MA', 'NH', 'RI', 'VT', 'NJ', 'NY', 'PA'],
    'II-Medio Oeste': ['IN', 'IL', 'MI', 'OH', 'WI', 'IA', 'KA', 'MN', 'MO', 'Ne', 'ND', 'SD'],
    'III-Sur': ['DE', 'DC', 'FL', 'GA', 'MD', 'NC', 'SC', 'VA', 'WV', 'AL', 'KY', 'MS', 'TN', 'AR', 'LA', 'OK', 'TX'],
    'IV-Oeste': ['AZ', 'CO', 'ID', 'NM', 'MT', 'UT', 'NV', 'WY', 'AK', 'CA', 'HI', 'OR', 'WA' ]
}

In [ ]:
# Función para filtrar por región
def filtrar_por_region(ddf, region):
    estados = regiones[region]
    return ddf[ddf['state'].isin(estados)]

In [ ]:
# Filtrar los datos por cada región y guardar los resultados
for region, estados in regiones.items():
    # Filtrar y guardar
    df_region = filtrar_por_region(ddf, region)
    df_region.compute().to_csv(f'restaurantes_{region}.csv', index=False)

In [ ]:
# Combinar todos los resultados en un solo DataFrame 
ddf_combinado = dd.concat([filtrar_por_region(ddf, region) for region in regiones])
ddf_combinado.compute().to_csv('restaurantes_todos.csv', index=False)

### Visualización

In [ ]:
region_1 = pd.read_csv('restaurantes_I-Noreste.csv')

In [ ]:
region_1

In [ ]:
region_2 = pd.read_csv('restaurantes_II-Medio Oeste.csv')

In [ ]:
region_2

In [ ]:
region_3 = pd.read_csv('restaurantes_III-Sur.csv')

In [ ]:
region_3

In [ ]:
region_4 = pd.read_csv('restaurantes_IV-Oeste.csv')

In [ ]:
region_4

### Consideraciones

Si se quiere ordenar por región (no probado)

Aunque se concatena los DataFrames en el orden en que se filtra, el orden de las filas en el DataFrame resultante no  garantiza que sea exactamente el mismo que el orden en que se agregaron los DataFrames individuales.
Si necesitas garantizar que las filas en el DataFrame combinado estén en un orden específico se debes ordenar el DataFrame resultante después de la concatenación.

In [ ]:
# Se guardan los datos ordenados por región en un archivo .csv
restaurantes_regiones = pd.concat([region_1, region_2, region_3, region_4], ignore_index=True)

# Ordenar por región (asumiendo que tienes una columna 'region' en tu DataFrame)
restaurantes_regiones = restaurantes_regiones.sort_values(by='region')

# Guardar el DataFrame combinado en un archivo CSV
restaurantes_regiones.to_csv('restaurantes_regiones.csv', index=False)